# Day 11 - 2D Cellular automatons

- https://adventofcode.com/2020/day/11

This is familiar territory for me, as we covered similar systems in years before (see days [12](../2018/Day%2012.ipynb) and [18](../2018/Day%2018.ipynb) in 2018, and days [11](../2019/Day%2011.ipynb) and [24](../2019/Day%2024.ipynb) in 2019).

Like before, using numpy, [`scipy.signal.convolve2d()`](https://docs.scipy.org/doc/scipy-1.5.4/reference/generated/scipy.signal.convolve2d.html), and [`numpy.select()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.select.html) make it really easy to generate each state.

I actually discovered a bug in my original Day 18 code: I had _transposed_ the input map. That was fine then, but for this puzzle I couldn't figure out why my puzzle would not converge until I minutely compared my puzzle input with the matrix produced. Oops.


In [1]:
from collections.abc import Mapping
from enum import Enum

import numpy as np
from scipy.signal import convolve2d


class Seat(Enum):
    floor = 0, "."
    empty = 1, "L"
    occupied = 2, "#"

    def __new__(cls, int_: int, value: str):
        instance = object.__new__(cls)
        instance._value_ = value
        instance.int = int_
        return instance


_kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])


class WaitingArea:
    def __init__(self, waiting_area_map: str) -> None:
        self._matrix = np.array(
            [Seat(c).int for line in waiting_area_map.splitlines() for c in line]
        ).reshape((-1, waiting_area_map.index("\n")))

    def __str__(self) -> str:
        mapping = {s.int: s.value for s in Seat}
        return "\n".join(
            ["".join(map(mapping.__getitem__, row)) for row in self._matrix]
        )

    @property
    def occupied(self) -> int:
        return np.sum(self._matrix == Seat.occupied.int)

    @property
    def counts(self) -> Mapping[Seat, "np.array[np.int]"]:
        f = self._matrix
        return {
            seat: convolve2d(f == seat.int, _kernel, mode="same")
            for seat in Seat
            if seat is not Seat.floor
        }

    def run(self, min_occupied_count: int = 4) -> int:
        """Run until stability is reached, the return the number of occupied seats"""
        f = self._matrix
        full = {s: np.full(f.shape, s.int) for s in Seat if s is not Seat.floor}
        while True:
            counts = self.counts
            rules = {
                # If a seat is **empty** (L) and there are **no** occupied seats adjacent to
                # it, the seat becomes **occupied**.
                Seat.occupied: (f == Seat.empty.int) & (counts[Seat.occupied] == 0),
                # If a seat is **occupied** (#) and **four or more** seats adjacent to it are
                # also occupied, the seat becomes **empty**.
                Seat.empty: (f == Seat.occupied.int)
                & (counts[Seat.occupied] >= min_occupied_count),
            }
            f = np.select(list(rules.values()), [full[s] for s in rules], default=f)
            if np.array_equal(self._matrix, f):
                return self.occupied
            self._matrix = f


test_map = """\
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""
assert WaitingArea(test_map).run() == 37

In [2]:
import aocd

data = aocd.get_data(day=11, year=2020)
area_map = WaitingArea(data)

In [3]:
print("Part 1:", area_map.run())

Part 1: 2438


## Part 2, toss out convolve2d

Part two ups the stakes: instead of a convenient _8 seats directly around the current_ reference frame which is the same for all seats in the waiting room, each seat in the waiting room now has a _custom_ mask. Where before we could use `convolve2d` to do all the counting in a single step, we now have to find a new way of counting surrounding occupied seats.

We first need to build a per-seat mask, a boolean matrix that has `True` values in the locations of the other seats. We can then use those masks to produce the same output that `convolve2d` would. You duplicate the current matrix `len(matrix)` times, then use the per-seat mask against that to select the neighbouring seats for every input seat and sum that back to summed counts per coordinate.


In [4]:
from functools import cached_property
from itertools import product

_directions = [(x, y) for x, y in product(range(-1, 2), repeat=2) if x or y]


class ImprovedWaitingArea(WaitingArea):
    @cached_property
    def _visible_seats(self) -> "np.array[bool]":
        # build visible seats maps
        f = self._matrix
        vs = np.zeros(f.shape * 2, dtype=bool)
        it = np.nditer(f, flags=["multi_index"])
        for v in it:
            if v == Seat.floor.int:
                continue
            nmap = vs[it.multi_index]
            # walk in each of the 8 directions; if we hit a seat, that's the visible
            # neighbour.
            for dx, dy in _directions:
                x, y = it.multi_index[0] + dx, it.multi_index[1] + dy
                while 0 <= x < nmap.shape[0] and 0 <= y < nmap.shape[1]:
                    if f[x, y] != Seat.floor.int:
                        nmap[x, y] = True
                        break
                    x, y = x + dx, y + dy
        return vs

    @property
    def counts(self):
        f, vs = self._matrix, self._visible_seats
        current = np.repeat(f[np.newaxis, ...], f.size, axis=0).reshape(f.shape * 2)
        return {
            seat: np.sum((current == seat.int) & vs, (2, 3))
            for seat in Seat
            if seat is not Seat.floor
        }

    def run(self) -> int:
        return super().run(5)


assert ImprovedWaitingArea(test_map).run() == 26

In [5]:
improved_area_map = ImprovedWaitingArea(data)
print("Part 2:", improved_area_map.run())

Part 2: 2174
